# CDC Setup for RiotX Integration
This notebook sets up Change Data Capture (CDC) environment with roles, permissions, and sample data for RiotX integration.

## 1. Role and Permissions Setup
Create a dedicated CDC role with minimal required permissions for RiotX integration

In [ ]:
CREATE OR REPLACE ROLE riotx_cdc
COMMENT = 'minimum cdc role for riotx';

-- Grant warehouse access for compute operations
GRANT USAGE, OPERATE ON WAREHOUSE compute_wh TO ROLE riotx_cdc;

## 2. Database and Schema Creation
Set up the main database and schemas for raw POS data and CDC operations

In [ ]:
CREATE DATABASE IF NOT EXISTS tb_101;
USE DATABASE tb_101;

CREATE OR REPLACE SCHEMA tb_101.raw_pos;
CREATE OR REPLACE SCHEMA tb_101.raw_pos_cdc;

-- Grant basic database and schema access to the CDC role
GRANT USAGE ON DATABASE tb_101 TO ROLE riotx_cdc;
GRANT USAGE ON SCHEMA tb_101.raw_pos TO ROLE riotx_cdc;
GRANT USAGE ON SCHEMA tb_101.raw_pos_cdc TO ROLE riotx_cdc;

## 3. CDC-Specific Permissions
Grant permissions needed for Change Data Capture operations

In [ ]:
GRANT SELECT ON FUTURE TABLES IN SCHEMA tb_101.raw_pos_cdc TO ROLE riotx_cdc;
GRANT CREATE TABLE ON SCHEMA tb_101.raw_pos_cdc TO ROLE riotx_cdc;
GRANT CREATE STREAM ON SCHEMA tb_101.raw_pos_cdc TO ROLE riotx_cdc;
GRANT SELECT ON FUTURE STREAMS IN SCHEMA tb_101.raw_pos_cdc TO ROLE riotx_cdc;

## 4. User Creation and Role Assignment
Create the RiotX CDC user with appropriate defaults and role assignments

In [ ]:
CREATE OR REPLACE USER riotx_cdc
    DEFAULT_ROLE = 'riotx_cdc'
    DEFAULT_WAREHOUSE = 'compute_wh'
    PASSWORD = '{{PASSWORD}}';

-- Assign roles to enable proper access hierarchy
GRANT ROLE riotx_cdc TO ROLE accountadmin;
GRANT ROLE riotx_cdc TO USER riotx_cdc;
GRANT ROLE accountadmin TO USER riotx_cdc;

## 5. File Format and Stage Setup
Configure CSV file format and S3 stage for data loading

In [ ]:
CREATE OR REPLACE FILE FORMAT tb_101.public.csv_ff
type = 'csv';

CREATE OR REPLACE STAGE tb_101.public.s3load
COMMENT = 'Quickstarts S3 Stage Connection'
URL = 's3://sfquickstarts/frostbyte_tastybytes/'
FILE_FORMAT = tb_101.public.csv_ff;

## 6. Main Table Creation and Data Load
Create the primary order_header table and load initial data from S3

In [ ]:
CREATE OR REPLACE TABLE tb_101.raw_pos.order_header
(
    order_id NUMBER(38,0),
    truck_id NUMBER(38,0),
    location_id FLOAT,
    customer_id NUMBER(38,0),
    discount_id VARCHAR(16777216),
    shift_id NUMBER(38,0),
    shift_start_time TIME(9),
    shift_end_time TIME(9),
    order_channel VARCHAR(16777216),
    order_ts TIMESTAMP_NTZ(9),
    served_ts VARCHAR(16777216),
    order_currency VARCHAR(3),
    order_amount NUMBER(38,4),
    order_tax_amount VARCHAR(16777216),
    order_discount_amount VARCHAR(16777216),
    order_total NUMBER(38,4)
);

In [ ]:
-- Load data from S3 stage with size limit
COPY INTO tb_101.raw_pos.order_header
    FROM @tb_101.public.s3load/raw_pos/order_header/
    SIZE_LIMIT = 200000;

## 7. CDC Table Setup and Permissions
Create incremental table for CDC operations and enable change tracking

In [ ]:
CREATE OR REPLACE TABLE tb_101.raw_pos.incremental_order_header LIKE tb_101.raw_pos.order_header;

-- Enable change tracking for CDC functionality
ALTER TABLE tb_101.raw_pos.INCREMENTAL_ORDER_HEADER SET CHANGE_TRACKING = TRUE;

-- Grant read access to both tables for the CDC role
GRANT SELECT ON TABLE tb_101.raw_pos.order_header TO ROLE riotx_cdc;
GRANT SELECT ON TABLE tb_101.raw_pos.incremental_order_header TO ROLE riotx_cdc;

## 8. Initial Data Load and Testing
Load sample data into the incremental table and verify the setup

In [ ]:
INSERT INTO tb_101.raw_pos.incremental_order_header
 SELECT * FROM tb_101.raw_pos.order_header
 LIMIT 100 OFFSET 0;

In [ ]:
-- Verify initial data load
SELECT * FROM tb_101.raw_pos.incremental_order_header;

## 9. Additional Data Load
Add more sample data to simulate incremental updates

In [ ]:
INSERT INTO tb_101.raw_pos.incremental_order_header
 SELECT * FROM tb_101.raw_pos.order_header
 LIMIT 1000 OFFSET 100;